## Prerequisites

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns

In [ ]:
def display_data(fake_data, train_data):
    fake_data = pd.DataFrame(np.array(fake_data))
    fake_data.head()

    train_data = pd.DataFrame(np.array(train_data))
    train_data.head()

    fake_data["label"] = 1
    train_data["label"] = 0
    
    data = pd.concat([fake_data, train_data])
    data.head()

    sns.pairplot(data, hue='label')
    
def norm(x, mu, theta):
    return (x - mu)/theta

def denorm(x, mu, theta):
    return x * theta + mu

## Observation of data

Let's see how our data look like

In [ ]:
df1 = pd.read_csv('/home/train.csv', header=None)
df2 = pd.read_csv('/home/data_val_log_return.csv', header=None)
df = pd.concat([df1, df2])

train_data = np.array(df.drop(labels=0, axis=1))
data_dim = train_data.shape[1]

### Definition of GAN models

In [ ]:
import torch
import torch.nn as nn

if torch.cuda.is_available():
    print('You use GPU !')
    device = torch.device('cuda')
else:
    print('You use CPU !')
    device = torch.device('cpu')

You use GPU !


In [ ]:
# Generative model
def make_generator(noise_dim):
    return nn.Sequential(
        nn.Conv1d(in_channels=noise_dim, out_channels=128, kernel_size=1),
        nn.BatchNorm1d(128),
        nn.LeakyReLU(),
        
        nn.Conv1d(in_channels=128, out_channels=64, kernel_size=1),
        nn.BatchNorm1d(64),
        nn.LeakyReLU(),
        
        nn.Flatten(),
        
        nn.Linear(in_features=64, out_features=32),
        nn.Linear(in_features=32, out_features=16),
        nn.Linear(in_features=16, out_features=data_dim),
        nn.LeakyReLU(),
    )


# Discriminative model
def make_discriminator():
    return nn.Sequential(
        nn.ConvTranspose1d(in_channels=data_dim, out_channels=64, kernel_size=1),
        nn.BatchNorm1d(64),
        nn.LeakyReLU(),
        nn.Dropout(0.2),
        
        nn.ConvTranspose1d(in_channels=64, out_channels=16, kernel_size=1),
        nn.BatchNorm1d(16),
        nn.LeakyReLU(),
        nn.Dropout(0.1),
        
        nn.Flatten(),
        nn.Linear(in_features=16, out_features=8),
        
        nn.Linear(in_features=8, out_features=2)
    )

In [ ]:
#function to test multiple classical gan architectures

## each network will be a combinason of these given list function
#  - MLP
#  - CONV1d
#  - 

#def test_clasical(nb)

## Training

Split dataset into training data and evaluation data

In [ ]:
train_data.shape

(1156, 4)

In [ ]:
X = torch.tensor(train_data , dtype=torch.float).to(device) ** 0.25
mu = X.mean(dim=0)
theta = torch.sqrt(((X - mu)**2).mean(dim=0))
X = norm(X, mu, theta)

eval_rate = 0.2
eval_index = int(X.shape[0] * (1 - eval_rate))
X_train = X[0: eval_index]
X_eval = X[eval_index:]

batch_size = 16

X_train_loader = torch.utils.data.DataLoader(X_train, batch_size, shuffle=True)
# X_eval_loader = torch.utils.data.DataLoader(X_eval, batch_size, shuffle=True)

We choose Adam Optimizer ($\beta_1 = 0.9$, $\beta_2 = 0.999$)

In [ ]:
noise_dim = 200
generator = make_generator(noise_dim=noise_dim).to(device)
discriminator = make_discriminator().to(device)


num_epochs = 100
lr = 0.001
gen_optimizer = torch.optim.Adam(generator.parameters(), lr=lr)
disc_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr)

criterion = torch.nn.CrossEntropyLoss()

In [ ]:
def reshape(data):
    return data.view(data.shape[0], data.shape[1], 1)

In [ ]:
tk = tqdm(range(num_epochs))

for i in tk:
    disc_losses = []
    gen_losses = []
    for true_data in X_train_loader:
        #### DISCRIMINATOR OPTMIZATION ####
        n_data = true_data.shape[0]
        
        discriminator.train() # Unfreeze parameters of the discriminator
        generator.eval() # Freeze parameters of the generator
        
        disc_optimizer.zero_grad()
        
        true_x = true_data.to(device) # true data from the training dataset
        
        noise = torch.randn(n_data, noise_dim).to(device)
        noise = reshape(noise)
        
        fake_x = generator(noise).detach() # fake data from the noise distribution ~ N(0, 1)

        x = torch.cat([true_x, fake_x]) # Gather true and fake data
        x = reshape(x)
        
        
        true_y = torch.ones((n_data,), dtype=torch.long).to(device) # target 1 for true data
        fake_y = torch.zeros((n_data,), dtype=torch.long).to(device) # target 0 for fake data
        
        y = torch.cat([true_y, fake_y]) # Gather true and fake targets
        
        
        
        output = discriminator(x)
        
        disc_loss = criterion(output, y) # Penalize Discriminator for failing to distinguish fake data from true data
        disc_losses.append(disc_loss.item())
        
        disc_loss.backward()
        
        
        disc_optimizer.step() # Optimize Discriminator
        
        
        
        ### GENERATOR OPTIMIZATION ###
        n_data = true_data.shape[0] * 2
    
        discriminator.eval() # Freeze parameters of the discriminator
        generator.train() # Unfreeze parameters of the generator

        gen_optimizer.zero_grad()
        
        
        noise = torch.randn(n_data, noise_dim).to(device)
        noise = reshape(noise)
        fake_x = generator(noise) # fake data from the noise distribution ~ N(0, 1)
        
        true_y = torch.ones((n_data,), dtype=torch.long).to(device) # target 1 for true data
        
        fake_x = reshape(fake_x)
        output = discriminator(fake_x)
        
        gen_loss = criterion(output, true_y) # Penalize Generator for failing to fool the discriminator
        gen_losses.append(gen_loss.item())
        
        gen_loss.backward()
        

        gen_optimizer.step() # Optimize Generator  
    
    
    disc_loss = np.mean(disc_losses)
    gen_loss = np.mean(gen_losses)
    
    tk.set_postfix({"disc_loss": disc_loss, "gen_loss": gen_loss})

  0%|          | 1/500 [00:01<09:04,  1.09s/it, disc_loss=0.331, gen_loss=2.01]


KeyboardInterrupt: ignored

In [ ]:
discriminator.eval()
generator.eval()

Sequential(
  (0): Conv1d(200, 128, kernel_size=(1,), stride=(1,))
  (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): LeakyReLU(negative_slope=0.01)
  (3): Conv1d(128, 64, kernel_size=(1,), stride=(1,))
  (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=64, out_features=32, bias=True)
  (8): Linear(in_features=32, out_features=16, bias=True)
  (9): Linear(in_features=16, out_features=4, bias=True)
  (10): LeakyReLU(negative_slope=0.01)
)

In [ ]:
example = torch.randn(batch_size*4, noise_dim).to(device)
example = reshape(example)

generated = generator(example)
generated = reshape(generated)

discriminator(generated).argmax(dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
        0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0])

In [ ]:
probs = discriminator(generated)
probs[:10]

tensor([[ 1.3450, -0.9090],
        [ 0.7438, -0.2529],
        [ 0.7613, -0.2703],
        [ 1.5110, -1.0501],
        [ 0.6505, -0.1776],
        [ 1.3322, -0.8807],
        [ 0.4257,  0.1054],
        [ 1.4165, -0.9702],
        [ 1.0844, -0.6233],
        [ 0.9545, -0.4886]], grad_fn=<SliceBackward0>)

In [ ]:
x = reshape(X)
probs = discriminator(x) # .argmax(dim=1)[:300]
probs[:10]

tensor([[ 0.6427, -0.1768],
        [-0.1275,  0.5120],
        [-5.7178,  7.4319],
        [ 0.3806,  0.0650],
        [ 0.8752, -0.4333],
        [-2.9503,  3.8509],
        [ 1.1600, -0.7192],
        [ 0.4996,  0.0337],
        [-2.9724,  3.8570],
        [-1.4818,  2.0946]], grad_fn=<SliceBackward0>)

In [24]:
fake_data = generator(reshape(torch.randn(X_eval.shape[0], noise_dim).to(device)))
fake_data = denorm(fake_data, mu, theta) ** 4
fake_data = fake_data.detach().cpu().numpy()

display_data(fake_data, X_eval)

ValueError: ignored

In [ ]:
fake_data[:10]

In [ ]:
def absolute_kendall_error(data, generated):
    scores = []
    for i in range(data.shape[1]):
        data_i = data[:, i]
        generated_i = generated[:, i]
        # 6.27272727, 6.63636364, 7.72727273, 9.        , 9.72727273
        count_data_i = [np.sum(np.array(
            list(data_i[:j]) + list(data_i[j+1:])
        ) < data_i[j]) for j in range(len(data_i))]
        
        count_gen_i = [np.sum(np.array(
            list(generated_i[:j]) + list(generated_i[j+1:])
        ) < generated_i[j]) for j in range(len(generated_i))]
        
        count_data_i = np.array(count_data_i) / len(count_data_i)
        count_gen_i = np.array(count_gen_i) / len(count_gen_i)
        
        error = sum(abs(d-g) for d,g in zip(count_data_i, count_gen_i))
        error /= (len(data_i))
        
        scores.append(error)
    return scores

In [ ]:
absolute_kendall_error(train_data, fake_data)

In [ ]:
np.mean(absolute_kendall_error(train_data, fake_data))